# Análisis de datos financieros

En finanzas, la creación de un portafolio de inversiones que se adecúe a las necesidades y perfil de una persona es de gran importancia, no sólo normativa si no para que duerma un poco más tranquila por las noches.

Para eso, es necesario revisar muchas variables, desde precios de activos, hasta cotizaciones de monedas o variables macroeconómicas. En este primer práctico vamos a entender un poco más sobre los precios de los activos, en particular de las acciones, y algunas formas de visualizarlos.

Empecemos con la obtención de datos, a partir de lo que vimos en el Notebook Introductorio.

In [ ]:
from pandas_datareader import data

In [ ]:
# Un ticker es la abreviación con la cual se encuentra al activo
# cuando se busca su precio o se realiza una orden de mercado.
tickers = ['MELI']

# Definimos el rango de fechas deseado
start_date = '2020-01-01'
end_date = '2020-12-31'

# Obtenemos un DataFrame
df = data.DataReader(
    tickers,
    start=start_date, 
    end=end_date, 
    data_source='yahoo',
)

# Cómo seguimos?

Una **acción** es un instrumento que representa la propiedad de una fracción de una empresa. Esto nos da propiedad de una proporción de los activos y sus ganancias (o pérdidas) de acuerdo a cuántas unidades se posee. Es posible comprarlas y venderlas en mercados como BYMA en Argentina o el NYSE en USA (teniendo una cuenta comitente).

Por lo general, un buen recurso para aprender sobre instrumentos y estrategias de inversión en el mercado es la página de [Investopedia](https://www.investopedia.com).

Las acciones que descargamos aquí arriba son las de Mercado Libre en USA.

In [ ]:
df.head()

## Primer Acercamiento
En este caso, descargamos los precios DIARIOS de la acción de Mercado Libre. Los mismos vienen en 6 columnas. Por ahora podemos ignorar la que se llama *Adj Close* y vamos a utilizar las otras cinco para realizar nuestro primer análisis (Podemos hacer un drop de la columna que no usaremos).

El mercado está abierto durante los días hábiles y, para cada uno de esos días, tenemos los siguientes datos:
- el precio al cual comenzó a operar la acción, o apertura (Open),
- el máximo alcanzado en el día (High),
- el mínimo (Low),
- el precio al cual se operó por última vez la acción, o cierre (Close) y,
- la cantidad de acciones que se intercambiaron en el día (Volume)

Como primera consigna, puede ser de utilidad conseguir algunas estadísticas descriptivas de este dataset.

In [ ]:
# Describir el dataset

## Gráficos de Vela

![Gráfico de Vela](https://media.istockphoto.com/vectors/stock-exchange-vector-id867941110?k=6&m=867941110&s=612x612&w=0&h=zE-LBIx2hT0eNEfDnuk1EQsGuWplf_hP76ETagH_vic=)

Los gráficos de vela son una buena forma de visualizar la evolución de los precios de un instrumento en cualquier intervalo de tiempo. La forma de leerlos es la siguiente:

- Cada barra comprime la información de un período (en nuestro caso puede ser cada fila).
- Si el precio de cierre es mayor al de apertura, el color de la vela es verde, caso contrario es rojo y el tope y base de la barra denotan esos precios.
- El máximo alcanzado en el período es la parte superior de la línea y el mínimo es la parte más baja de la línea.
- Algunos gráficos, además, tienen una visualización inferior en forma de barras para el volumen operado de cada período.

Podemos hacer un gráfico de vela con el dataframe que creamos? Queda bastante lindo (e interactivo!) si lo hacemos con la librería [Plotly](https://plot.ly/python/candlestick-charts/).

## Retornos Periódicos

Si bien es útil entender el comportamiento de un activo a través de sus precios, los precios no son comparables de forma directamente entre diferentes activos. Dado que son valores nominales que dependen de una serie de eventos corporativos, poco podemos decir respecto a dos compañias solo porque una cotice a 1 dólar y otra a 100.

Lo importante en finanzas es conocer los RETORNOS o rendimientos de los activos y el primer paso para hacerlo es ver cuánto varía el precio de un activo desde un cierre hasta el próximo. Para eso existe el muy útil método [pct_change](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html).

Vamos a ver qué tan útil termina siendo esto:
- Grafiquemos la serie de tiempo (solamente plot de Pandas) de los precios de Close para dos acciones.
- Hagamos lo mismo para los retornos de las acciones.

Parece que va a ser más sencillo revisar algo en este espacio de retornos:

- Veamos estadísticas básicas (media, desviación estándar, moda).
- Hagamos un distplot de la distribución de los retornos.
- Una afirmación que se suele hacer con los retornos diarios de activos financieros, es que se asemejan bastante a una distribución normal. Es así en este caso?


## Volatilidad de un activo financiero

A veces, además de saber cuánto dinero se ganó o perdió con algún activo financiero, también es de utilidad saber qué tan volátil fue ese retorno. Algunos activos tienen un buen rendimiento a lo largo del tiempo, pero a cambio de que su precio experimente subidas y bajadas muy pronunciadas. Para muchas personas, esto puede ser inaceptable, porque no todas tienen una idea de cuánto tiempo pueden dejar su dinero invertido en una acción o si lo necesitarán algún día en caso de emergencia.

Una buena medida de la volatilidad de un activo financiero es la **desviación estándar de sus retornos**. Podemos intentar graficar la media de los retornos vs. su volatilidad en un scatterplot. Podemos hacerlo para muchas acciones diferentes, o para ventanas de tiempo de una misma acción (usando [Rolling](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html) para nuestro DataFrame). 

## Sharpe Ratio

Este cociente fue ideado por William Sharpe (economista y ganador del Premio Nobel) para ayudar a entender el retorno de una inversión comparado con el riesgo de la misma. Se puede conseguir dividiendo el exeso de retorno (la diferencia entre el retorno del activo y el de un activo libre de riesgo) de una inversión por su volatilidad. En una primera aproximación podemos asumir que el retorno libre de riesgo es Cero y simplemente tomamos el cociente rendimiento / riesgo.

Este cociente se lee como "unidades de retorno que brinda un activo por cada unidad de riesgo asumida". De esa forma se obtiene un indicador que permite comparar activos de diferentes niveles de retorno y riesgo en un mismo espacio, y es una buena adición al gráfico de retorno vs. volatilidad como su "hue".

Valores altos de Sharpe Ratio siempre son buscados, porque representan una muy buena relación retorno-riesgo para una inversión, pero una de las "debilidades" que tiene es que se asume que los retornos tienen una distribución normal (esto se cumple?). Aún así, es una métrica muy utilizada en el mundo para evaluar la fortaleza tanto de activos individuales, como conjuntos de activos e incluso estrategias de trading.

Cómo podríamos modificar el scatterplot anterior para agregarlo?